In [178]:
import random
import math
random.seed(42)

In [179]:
class City():
    def __init__(self, x=None, y=None):
        if x is None:
            x=random.random() * 200
        if y is None:
            y=random.random() * 200
        
        self.x = x
        self.y = y
        
    def distanceTo(self,city):
        x_distance = abs(self.x - city.x)
        y_distance = abs(self.y - city.y)
        total_distance = math.sqrt(pow(x_distance, 2) + pow(y_distance, 2))
        return total_distance

In [180]:
class TourManager():
    
    destination_cities = []
    
    @classmethod
    def addCity(self,city):
        self.destination_cities.append(city)
    
    @classmethod
    def getCity(self,index):
        return self.destination_cities[index]
    
    @classmethod
    def numberCities(self):
        return len(self.destination_cities)

In [181]:
class Tour():
    def __init__(self):
        self.tour = [None] * TourManager.numberCities()
        self.fitness = 0
        self.distance = 0
        
    def generateIndividual(self):
        
        
        for i in range (0, TourManager.numberCities()):
            
            self.setCity(i, TourManager.getCity(i))
        
        random.shuffle(self.tour)
        
    def getCity(self,index):
        return self.tour[index]
    
    def setCity(self,index, city):
        self.tour[index] = city
        
        self.fitness = 0
        self.distance = 0
        
    def getFitness(self):
        if self.fitness==0:
            self.fitness = 1 / self.getDistance()
            
        return self.fitness
    
    def getDistance(self):
        if self.distance==0:
            tourDistance = 0
            
            for i in range(0, self.tourSize()):
                fromCity = self.getCity(i)
                if i+1 < self.tourSize():
                    destinationCity = self.getCity(i+1)
                else:
                    destinationCity = self.getCity(0)
            
                tourDistance += fromCity.distanceTo(destinationCity)
        
            self.distance = tourDistance
        
        return self.distance
    
    def containsCity(self,city):
        if city in self.tour:
            return True
        else:
            return False
        
    def tourSize(self):
        return len(self.tour)
    
    def prettyPrint(self):
        geneString = "|"
        for i in range(0, self.tourSize()):
            geneString += str(int(self.getCity(i).x)) + "," + str(int(self.getCity(i).y))  + "|"
        
        return geneString
        

In [182]:
class Population():
    def __init__(self, populationSize, initialise):
        self.populationSize = populationSize
        self.initialise = initialise
        self.tours = [None] * populationSize
                
        if initialise:
            for i in range(0, self.populationSize):
                tour = Tour()
                
                tour.generateIndividual() #issue
                self.saveTour(i, tour)
    
    def saveTour(self, index, tour):
        self.tours[index] = tour
        
    def getTour(self, index):
        return self.tours[index]
    
    def getFittest(self):
        fittest = self.tours[0]
        for i in range(1, self.populationSize):
            if fittest.getFitness() <= self.getTour(i).getFitness():
                fittest = self.getTour(i)
                
        return fittest
    
    def populationSize(self):
        return len(self.tours)

In [183]:
class GA():
    mutationRate = 0.015
    tournamentSize = 5
    elitism = True
    
    @classmethod
    def evolvePopulation(self,pop):
        newPopulation = Population(pop.populationSize, False)
        
        elitismOffset = 0
        if self.elitism:
            newPopulation.saveTour(0, pop.getFittest())
            elitismOffset = 1
            
        for i in range(elitismOffset, newPopulation.populationSize):
            parent1 = self.tournamentSelection(pop)
            parent2 = self.tournamentSelection(pop)
            child = self.crossover(parent1, parent2)
            newPopulation.saveTour(i, child)
            
        for i in range(elitismOffset, newPopulation.populationSize):
            self.mutate(newPopulation.getTour(i))
        
        return newPopulation
    
    @classmethod
    def crossover(self, parent1, parent2):
        child = Tour()
        
        startPos = int(random.random() * parent1.tourSize())
        endPos = int(random.random() * parent2.tourSize())
        
        for i in range(0, child.tourSize()):
            if startPos < endPos and i > startPos and i < endPos:
                child.setCity(i, parent1.getCity(i))
            elif startPos > endPos:
                if not (i < startPos and i > endPos):
                    child.setCity(i, parent1.getCity(i))
                    
        for i in range(0, parent2.tourSize()):
            if not (child.containsCity(parent2.getCity(i))):
                for j in range(0, child.tourSize()):
                    if child.getCity(j) is None:
                        child.setCity(j, parent2.getCity(i))
                        break
        
        return child
    
    @classmethod
    def mutate(self, tour):
        for tourPos1 in range(0, tour.tourSize()):
            if random.random() < self.mutationRate:
                tourPos2 = int(random.random() * tour.tourSize())
                
                city1 = tour.getCity(tourPos1)
                city2 = tour.getCity(tourPos2)
                
                tour.setCity(tourPos2, city1)
                tour.setCity(tourPos1, city2)
    
    @classmethod
    def tournamentSelection(self,pop):
        tournament = Population(self.tournamentSize, False)
        
        for i in range(0, self.tournamentSize):
            randomId = int(random.random() * pop.populationSize)
            tournament.saveTour(i, pop.getTour(randomId))
            
        fittest = tournament.getFittest()
        return fittest
                

In [184]:
# Config
num_cities = 100
num_generations = 10000
population_size = 50

In [185]:
# Main
for i in range(0, num_cities):
    city = City()
    TourManager.addCity(city)

pop = Population(population_size, True)

In [186]:
print("Initial distance: " , pop.getFittest().getDistance())

pop = GA.evolvePopulation(pop)
for i in range(0, num_generations):
    pop = GA.evolvePopulation(pop)
    if i%100==0: print("Generation: ", i, " Fitness(Distance): " , pop.getFittest().getDistance())
    #print("Generation: ", i, " Fitness(Distance): " , pop.getFittest().getDistance())
print("Finished")
print("Final distance: " , pop.getFittest().getDistance())
print("Solution:")
print(pop.getFittest().prettyPrint());

Initial distance:  9630.731957929642
Generation:  0  Fitness(Distance):  9338.173901915512
Generation:  100  Fitness(Distance):  5408.396284590611
Generation:  200  Fitness(Distance):  4660.426951958268
Generation:  300  Fitness(Distance):  4192.749367559977
Generation:  400  Fitness(Distance):  3851.4175455833783
Generation:  500  Fitness(Distance):  3608.7994426400155
Generation:  600  Fitness(Distance):  3463.184696250255
Generation:  700  Fitness(Distance):  3339.48171015938
Generation:  800  Fitness(Distance):  3201.5120898623977
Generation:  900  Fitness(Distance):  3182.0088826093474
Generation:  1000  Fitness(Distance):  3075.198612065921
Generation:  1100  Fitness(Distance):  3029.4812263299336
Generation:  1200  Fitness(Distance):  2967.8299270704883
Generation:  1300  Fitness(Distance):  2936.9543317828957
Generation:  1400  Fitness(Distance):  2899.811430976218
Generation:  1500  Fitness(Distance):  2873.1407817278123
Generation:  1600  Fitness(Distance):  2848.080827854242